# Imports

In [1]:
import sqlite3
import pandas as pd
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt

SQL_LITE_DB_PATH = "D:\\Repos\\ut-health\\mimic-iii-gz\\mimic3.db"

# Connect to the SQLite database in the file at the given path d:\data\mimic.db
conn = sqlite3.connect(SQL_LITE_DB_PATH)

# print version of sqlite
cursor = conn.cursor()
cursor.execute("SELECT sqlite_version()")
print("SQLite version: %s" % cursor.fetchone())

# close connection
conn.close()

# print pandas version
print("Pandas version: %s" % pd.__version__)

# print matplotlib version
print(f"Matplotlib version: {matplotlib.__version__}")

print(f"NetworkX version: {nx.__version__}")

# Set pandas display options to show all columns and rows without truncation
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
print("\nRemoved truncation of columns")


SQLite version: 3.49.1
Pandas version: 2.2.3
Matplotlib version: 3.10.1
NetworkX version: 3.4.2

Removed truncation of columns


# Load Data

In [4]:
# load patient data
conn = sqlite3.connect("D:\\Repos\\ut-health\\mimic-iii-gz\\mimic3.db")

query = "SELECT * FROM patients"
patients_df = pd.read_sql_query(query, conn)

query = "SELECT * FROM d_icd_diagnoses"
d_icd_diagnoses_df = pd.read_sql_query(query, conn)

query = "SELECT * FROM diagnoses_icd"
diagnoses_icd_df = pd.read_sql_query(query, conn)

query = "SELECT * FROM admissions"
admissions_df = pd.read_sql_query(query, conn)

conn.close()

# Subselect size of data


In [ ]:
# print the number of patients and diagnoses
print(f"Number of patients: {len(patients_df)}")
print(f"Number of diagnoses: {len(diagnoses_icd_df)}")
print(f"Number of admissions: {len(admissions_df)}")

pt_frac = 0.0001  # 0.01% of the patients gives 5 patients, 52 diagnoses
pt_frac = 0.0002  # 0.02% of the patients gives 9 patients, 110 diagnoses
pt_frac = 0.0003  # 0.02% of the patients gives 14 patients, 188 diagnoses

# select 0.01% of the patients randomly
patients_df = patients_df.sample(frac=pt_frac, random_state=1)

# add DOB to admissions_df
admissions_df = admissions_df.merge(patients_df[['SUBJECT_ID', 'DOB']], left_on='SUBJECT_ID', right_on='SUBJECT_ID', how='inner', suffixes=('_adm', '_pt'))

# sort admissions_df by SUBJECT_ID and ADMITTIME in ascending order
admissions_df['ADMITTIME'] = pd.to_datetime(admissions_df['ADMITTIME'])
admissions_df = admissions_df.sort_values(by=['SUBJECT_ID', 'ADMITTIME'])

# keep the newest admission (HADM_ID) for each patient, drop the rest of the admissions
admissions_df = admissions_df.groupby('SUBJECT_ID').last().reset_index()

# keep only diagnoses that belong to HADM_IDs in admissions_df
diagnoses_icd_df = diagnoses_icd_df[diagnoses_icd_df['HADM_ID'].isin(admissions_df['HADM_ID'])]

# print the number of patients and diagnoses
print(f"Updated number of patients: {len(patients_df)}")
print(f"Updated number of diagnoses: {len(diagnoses_icd_df)}")
print(f"Updated number of admissions: {len(admissions_df)}")

# TODO: fix the HADM_ID issue, only keep diagnosies that are from the latest admissions under subject id, and drop the rest, do same for admissions_df

Number of patients: 46520
Number of diagnoses: 651047
Number of admissions: 58976
Updated number of patients: 14
Updated number of diagnoses: 188
Updated number of admissions: 14


# Feature Engineering

In [6]:
# log data
print(f"Number of patients: {len(patients_df)}")

# convert to integer from datetime type to calculate age (without overflow)
admissions_df['DOB'] = admissions_df['DOB'].astype('datetime64[s]').astype(int)
admissions_df['ADMITTIME'] = admissions_df['ADMITTIME'].astype('datetime64[s]').astype(int)

# calculate age at admission
admissions_df['AGE'] = (admissions_df['ADMITTIME'] - admissions_df['DOB']) / 86400 / 365

# create age intervals in 5 year increments, of type int
admissions_df['AGE_BUCKET'] = (admissions_df['AGE'] // 5).astype(int) * 5

# set age bucket to 95 where age is greater than 90
admissions_df.loc[admissions_df['AGE_BUCKET'] > 90, 'AGE_BUCKET'] = 95

# add AGE_BUCKET to patients_df
patients_df = patients_df.merge(admissions_df[['SUBJECT_ID', 'HADM_ID', 'AGE_BUCKET']], left_on='SUBJECT_ID', right_on='SUBJECT_ID', how='inner', suffixes=('_pt', '_adm'))

print(f"Updated number of patients (with age bucket): {len(patients_df)}")

Number of patients: 14
Updated number of patients (with age bucket): 14


In [7]:
patients_df.head(5)

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,HADM_ID,AGE_BUCKET
0,3850,4074,M,2121-08-01 00:00:00,2204-02-06 00:00:00,2204-02-06 00:00:00,2204-02-06 00:00:00,1,137421,80
1,44143,90889,M,2095-04-29 00:00:00,None,None,None,0,168745,55
2,39453,72753,M,2042-06-01 00:00:00,None,None,None,0,144029,85
3,37377,64908,M,2108-04-22 00:00:00,None,None,None,0,172851,75
4,38824,70273,F,2090-05-28 00:00:00,None,None,None,0,102912,75


# Create Graph

## Create patient nodes

In [8]:
# Create a graph
G = nx.Graph()

i = 0

for _, row in patients_df.iterrows():

    # Add a node with the patient ID as the node identifier and gender as an attribute, also add dob as an attribute
    G.add_node(row['SUBJECT_ID'], gender=row['GENDER'], age=row['DOB'], age_bucket=row['AGE_BUCKET'], hadm_id=row['HADM_ID'])

    i+=1
    if i % 2000 == 0:
        print(f"Added {i} patient nodes, percent: {i / len(patients_df) * 100:.2f}%")

# number of nodes in the graph
print(f"Number of nodes in the graph: {G.number_of_nodes()}")

Number of nodes in the graph: 14


## Create diagnosis to patient mappings

In [9]:
# use the diagnoses_icd_df to get the icd9 codes, group by icd9 code, and create a dictionary with the icd9 code as the key and the subject_id as the values
diagnoses_to_patients = diagnoses_icd_df.groupby('ICD9_CODE')['SUBJECT_ID'].apply(set).to_dict()

# print 1 random key and value from the dictionary
print(f"Sample keys and values from the dictionary:")
print(f"\t\t{list(diagnoses_to_patients.items())[12]}")
print(f"\t\t{list(diagnoses_to_patients.items())[15]}")

# print size of the dictionary and counts of values
print(f"Count of the number of keys (icd9s) in the dictionary: {len(diagnoses_to_patients)}")
print(f"Count of the number of values (subject_ids) in all keys: {sum(len(v) for v in diagnoses_to_patients.values())}")

unique_subject_ids = len(set([item for sublist in diagnoses_to_patients.values() for item in sublist]))

# count number of unique subject_ids across all values in the dictionary
print(f"Count of the number of unique subject_ids across all keys: {unique_subject_ids}")

# print number of patients with diagnosis 0389
print(f"Number of patients with diagnosis 0389: {len(diagnoses_to_patients.get('0389', []))}")

# assert that the number of unique subject_ids is equal to the number of patients in the patients_df dataframe
assert unique_subject_ids == len(patients_df), f"Number of unique subject_ids in the dictionary ({unique_subject_ids}) is not equal to the number of patients in the patients_df dataframe ({len(patients_df)})"


Sample keys and values from the dictionary:
		('1985', {49036})
		('25000', {72753, 17906})
Count of the number of keys (icd9s) in the dictionary: 149
Count of the number of values (subject_ids) in all keys: 188
Count of the number of unique subject_ids across all keys: 14
Number of patients with diagnosis 0389: 3


## Add diagnoses edges between patients

In [10]:
# print the number of edges in the graph
print(f"Number of edges: {len(G.edges())}")

# print the number of nodes in the graph
print(f"Number of nodes: {len(G.nodes())}")

Number of edges: 0
Number of nodes: 14


In [12]:
i=0
ONCE = True


# add diagnosis/icd9 edge between patients
for icd9_code, patients in diagnoses_to_patients.items():

    # get SHORT_TITLE from d_icd_diagnoses_df for the icd9 code
    short_title = d_icd_diagnoses_df[d_icd_diagnoses_df['ICD9_CODE'] == icd9_code]['SHORT_TITLE'].values

    if len(short_title) == 0:
        short_title = "Unknown"
    else:
        # convert to a string
        short_title = short_title[0]

    if ONCE:
        print(f"ICD9 code: {icd9_code}, short title: {short_title}")
        ONCE = False

    if len(patients) < 2:
        continue

    # if there are more than 1 patients with the same diagnosis, add edges between them
    patients = list(patients)

    combos = [(patients[i], patients[j]) for i in range(len(patients)) for j in range(i + 1, len(patients))]
    G.add_edges_from(combos, icd9=icd9_code, short_title=short_title)
    i += 1
    if i % 4000 == 0:
        print(f"Added {i} edges, percent: {i / len(diagnoses_to_patients) * 100:.2f}%")


ICD9 code: 0389, short title: Septicemia NOS


In [13]:
# print the number of edges in the graph
print(f"Number of edges: {len(G.edges())}")

# print the number of nodes in the graph
print(f"Number of nodes: {len(G.nodes())}")

Number of edges: 47
Number of nodes: 14


# Constants

In [14]:
import os

CURR_DIR_PATH = os.getcwd()

PICKLE_FILE_NAME = None
HTML_FILE_NAME = None

if G.number_of_nodes() < 10:
    PICKLE_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_10_nodes.gpickle"
    HTML_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_10_nodes.html"
elif G.number_of_nodes() < 50:
    PICKLE_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_50_nodes.gpickle"
    HTML_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_50_nodes.html"
elif G.number_of_nodes() < 100:
    PICKLE_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_100_nodes.gpickle"
    HTML_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_100_nodes.html"
elif G.number_of_nodes() < 1000:
    PICKLE_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_1000_nodes.gpickle"
    HTML_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_1000_nodes.html"
elif G.number_of_nodes() < 10000:
    PICKLE_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_10000_nodes.gpickle"
    HTML_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_10000_nodes.html"
else:
    PICKLE_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_46000_nodes.gpickle"
    HTML_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_46000_nodes.html"

print(f"Pickle file name: {PICKLE_FILE_NAME}")
print(f"HTML file name: {HTML_FILE_NAME}")

Pickle file name: d:\Repos\ut-health-final-proj\pickle\patients_graph_max_50_nodes.gpickle
HTML file name: d:\Repos\ut-health-final-proj\pyvis\graph_max_50_nodes.html


# Pyvis Graph

In [15]:
# use pyvis to draw the graph
from pyvis.network import Network


nt = Network(height="1200px", width="100%", notebook=False, directed=False, cdn_resources="remote")

print("created pyvis network")
nt.from_nx(G)

# for node in nt.nodes:
#     print(f"Node ID: {node['id']}")

# add node attributes to the pyvis network
for node in nt.nodes:
    node['title'] = f"Patient ID: {node['id']}\nGender: {node['gender']}\nAge Bucket: {node['age_bucket']}"
    node['label'] = f"{node['id']}"
    node['font'] = {'size': 5}

# add edge attributes to the pyvis network
for edge in nt.edges:
    edge['title'] = f"ICD9 Code: {edge['icd9']}"
    edge['label'] = f"{edge['short_title']}"
    # set font size to 5
    edge['font'] = {'size': 5}

print("added nx graph to pyvis network")
print(f"file name: {HTML_FILE_NAME}")
nt.toggle_physics(False)
nt.show_buttons()
nt.generate_html(HTML_FILE_NAME)
nt.save_graph(HTML_FILE_NAME)
# nt.show(HTML_FILE_NAME)
print("saved pyvis graph to file")


created pyvis network
added nx graph to pyvis network
file name: d:\Repos\ut-health-final-proj\pyvis\graph_max_50_nodes.html
saved pyvis graph to file


# Save Graph to Disk

## pickle

In [17]:
import pickle

# Save the graph to a pickle file
with open(f"{PICKLE_FILE_NAME}", "wb") as f:
    pickle.dump(G, f)
    print("Graph saved to pickle file")


Graph saved to pickle file


# Old

In [ ]:
pt_list = ["a", "b", "c"]

# create unique combos of patients
combos = [(pt_list[i], pt_list[j]) for i in range(len(pt_list)) for j in range(i + 1, len(pt_list))]
print(combos)


In [ ]:
# load prescription data
conn = sqlite3.connect("D:\\Repos\\ut-health\\mimic-iii-gz\\mimic3.db")

query = "SELECT * FROM admissions"
admissions_df = pd.read_sql_query(query, conn)

query = "SELECT * FROM d_labitems"
d_labitems_df = pd.read_sql_query(query, conn)

query = "SELECT * FROM labevents"
labevents_df = pd.read_sql_query(query, conn)

query = "SELECT * FROM prescriptions"
prescriptions_df = pd.read_sql_query(query, conn)

query = "SELECT * FROM d_items"
d_items_df = pd.read_sql_query(query, conn)

conn.close()
